In [ ]:
import os
import torch
import time
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader
from tqdm import tqdm

from model.dlinknet3 import DLinkNet34
from data.dataset import RoadDatasetLegacyAugment
from loss.loss import bce_dice_loss
from utils.early_stopping import EarlyStopping
from utils.save_load import save_model
from config.optimizer import get_optimizer, get_scheduler
from utils.metrics import compute_metrics

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# === Cấu hình ===
batch_size = 4
num_epochs = 50
lr = 1e-4
log_dir = 'logs'
model_path = 'checkpoints/dlinknet_model_best.pth'
csv_path = 'data/1024/split.csv'
image_dir = 'data/1024/images'
mask_dir = 'data/1024/masks'

In [4]:
# === Dataloader ===
train_dataset = RoadDatasetLegacyAugment(csv_path, image_dir, mask_dir, split='train', augment=True)
val_dataset = RoadDatasetLegacyAugment(csv_path, image_dir, mask_dir, split='val', augment=False)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

In [5]:
# === Model, Loss, Optimizer ===
model = DLinkNet34(num_classes=1).to(device)
optimizer = get_optimizer(model, lr)
scheduler = get_scheduler(optimizer, mode='min')
early_stopping = EarlyStopping(patience=10, mode='min')  # chuyển sang mode='max' nếu early stopping chọn theo dice

C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\torch\optim\lr_scheduler.py:62: UserWarn

In [6]:
# === Logger ===
writer = SummaryWriter(log_dir)
log_txt = open(os.path.join(log_dir, 'train_log.txt'), 'w')

In [7]:
# === Training Loop ===
best_dice = 0.0
for epoch in range(num_epochs):
    print(f"\n🟢 Epoch [{epoch+1}/{num_epochs}]")
    model.train()
    train_loss = 0

    for imgs, masks in tqdm(train_loader, desc='Training'):
        imgs, masks = imgs.to(device), masks.to(device)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = bce_dice_loss(outputs, masks)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    train_loss /= len(train_loader)

    # === Validation ===
    model.eval()
    val_loss = 0
    all_preds = []
    all_masks = []

    with torch.no_grad():
        for imgs, masks in tqdm(val_loader, desc='Validation'):
            imgs, masks = imgs.to(device), masks.to(device)
            outputs = model(imgs)
            loss = bce_dice_loss(outputs, masks)
            val_loss += loss.item()

            all_preds.append(outputs.cpu())
            all_masks.append(masks.cpu())

    val_loss /= len(val_loader)
    all_preds = torch.cat(all_preds)
    all_masks = torch.cat(all_masks)

    val_dice, val_iou, val_miou = compute_metrics(all_preds, all_masks, threshold=0.5)

    print(f"🔹 Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Dice: {val_dice:.4f} | IoU: {val_iou:.4f} | mIoU: {val_miou:.4f}")
    log_txt.write(f"Epoch {epoch+1}: train_loss={train_loss:.4f}, val_loss={val_loss:.4f}, dice={val_dice:.4f}, iou={val_iou:.4f}, mIoU: {val_miou:.4f}\n")

    # === TensorBoard log ===
    writer.add_scalar('Loss/train', train_loss, epoch)
    writer.add_scalar('Loss/val', val_loss, epoch)
    writer.add_scalar('Dice/val', val_dice, epoch)
    writer.add_scalar('IoU/val', val_iou, epoch)
    writer.add_scalar('mIoU/val', val_miou, epoch)

    # === Scheduler + EarlyStopping ===
    scheduler.step(val_loss)
    early_stopping(val_loss, model)
    if early_stopping.early_stop:
        print("⛔ Early stopping triggered!")
        early_stopping.load_best_weights(model)
        break

    # === Save model if improved ===
    if val_dice > best_dice:
        best_dice = val_dice
        os.makedirs(os.path.dirname(model_path), exist_ok=True)
        save_model(model, model_path, epoch, best_dice)
        print("✅ Model improved. Saved.")

log_txt.close()
writer.close()
print("🏁 Training completed.")



🟢 Epoch [1/50]


Validation: 100%|██████████| 278/278 [00:29<00:00,  9.44it/s]


🔹 Train Loss: 0.6345 | Val Loss: 0.4631 | Dice: 0.6597 | IoU: 0.5112 | mIoU: 0.7404
✅ Model improved. Saved.

🟢 Epoch [2/50]


Validation: 100%|██████████| 278/278 [00:29<00:00,  9.39it/s]


🔹 Train Loss: 0.4757 | Val Loss: 0.4308 | Dice: 0.6807 | IoU: 0.5342 | mIoU: 0.7534
✅ Model improved. Saved.

🟢 Epoch [3/50]


...

🔹 Train Loss: 0.3168 | Val Loss: 0.3417 | Dice: 0.7422 | IoU: 0.6120 | mIoU: 0.7956

🟢 Epoch [49/50]


Validation: 100%|██████████| 278/278 [00:29<00:00,  9.44it/s]


🔹 Train Loss: 0.3170 | Val Loss: 0.3381 | Dice: 0.7450 | IoU: 0.6147 | mIoU: 0.7970
✅ Model improved. Saved.

🟢 Epoch [50/50]


Validation: 100%|██████████| 278/278 [00:29<00:00,  9.44it/s]


🔹 Train Loss: 0.3146 | Val Loss: 0.3423 | Dice: 0.7424 | IoU: 0.6123 | mIoU: 0.7959
🏁 Training completed.
